#### Analyzing sar data programatically:

In [1]:
import os
import sys
parentDir = os.path.abspath('..')
sys.path.append(parentDir)

In [2]:
# This used to be necessary in prior version. Not anymore with the recent repackaging
# from sar.sar import sar
# from utils.table import Table
# from utils.BarPlot import BarPlot
# from utils.ListTables import printList, getListDisplayText, printStreamBuffer, stack, underline
# from utils.InfoRecord import Stage
# import json

In [3]:
from sar import sar
from sar.utils import Table
from sar.utils import BarPlot
from sar.utils import printList, getListDisplayText, printStreamBuffer, stack, underline
from sar.utils import Stage
import json

In [4]:
import glob
sarfiles = glob.glob( './sar*txt')

In [5]:
sarfiles

['./saru.txt',
 './sarb.txt',
 './sarw.txt',
 './sarr.txt',
 './sar-A.txt',
 './sar.txt']

In [6]:
fileDictionary={}
for fileName in sarfiles:
    with open (fileName) as fhand:
        fileDictionary[fileName]=sar(fhand)

In [7]:
fileDictionary

{'./saru.txt': <sar.sar.sar at 0x103ddfbc0>,
 './sarb.txt': <sar.sar.sar at 0x103ddda30>,
 './sarw.txt': <sar.sar.sar at 0x103daf890>,
 './sarr.txt': <sar.sar.sar at 0x103dad280>,
 './sar-A.txt': <sar.sar.sar at 0x10579e330>,
 './sar.txt': <sar.sar.sar at 0x103dad3a0>}

In [8]:
[(n, s.fetchtables('%user')) for n,s in fileDictionary.items()]

[('./saru.txt', []),
 ('./sarb.txt', []),
 ('./sarw.txt', []),
 ('./sarr.txt', []),
 ('./sar-A.txt', []),
 ('./sar.txt', [<sar.utils.table.Table at 0x1058de990>])]

In [9]:
[(n, s.fetchtables('%usr')) for n,s in fileDictionary.items()]

[('./saru.txt',
   <sar.utils.table.Table at 0x1058defc0>]),
 ('./sarb.txt', []),
 ('./sarw.txt', []),
 ('./sarr.txt', []),
 ('./sar-A.txt',
   <sar.utils.table.Table at 0x1058df290>]),
 ('./sar.txt', [])]

In [10]:
s1=fileDictionary['./sar.txt']
s1.tableList

[{(2864855809906397777, 'CPU', 'all'): <sar.utils.table.Table at 0x1058d7ad0>},
 {(2864855809906397777, 'CPU', 'all'): <sar.utils.table.Table at 0x1058de180>}]

In [11]:
for t in s1.fetchtables('%user'):
    t.print()

Timestamp       CPU     %user     %nice     %system     %iowait     %steal     %idle     
---------       ---     -----     -----     -------     -------     ------     -----     
12:11:00 AM     all     0.04      0.02      0.18        0.05        0.00       99.72     
12:20:08 AM     all     0.01      0.00      0.12        0.02        0.00       99.85     
12:30:15 AM     all     0.09      0.00      0.23        0.02        0.00       99.65     
12:40:07 AM     all     0.05      0.00      0.19        0.02        0.00       99.74     
12:50:15 AM     all     0.01      0.00      0.11        0.02        0.00       99.86     
01:00:15 AM     all     0.02      0.00      0.11        0.02        0.00       99.85     
01:10:08 AM     all     0.01      0.00      0.10        0.02        0.00       99.87     
04:02:49 AM     all     0.01      0.00      0.11        0.02        0.00       99.87     
04:36:35 AM     all     0.04      0.01      0.14        0.04        0.00       99.78     
04:46:50 A

In [12]:
def analyzeFields(fields, sarDataInstance):
    """analyzeFields runs once for every report.
    Each json field maps to a report name using "report" key. If a field specifies no report name, it will default to "Others"
    At this time, analyzeFields performs a sorting of each field referenced in sar.json using "sort" key.
    Fields are sorted in descending order - presuming the highest values are the ones of interest.
    A group column is displayed when specified in sar.json using "group" key."""
    lists=[]
    for field in fields:
        sort=field.get('sort')
        tables=sarDataInstance.fetchtables(sort)

        # A group is the field name for the "group by" column within a table
        # It is used to decide if the same field will be included in the table display.
        group=field.get('group',None)
        for t in tables:
            if group:
                data=BarPlot.appendBarPlot(t.get(('Timestamp',group,sort),filterFunc=lambda x:True, sortKey=lambda x:float(x(sort)), reverse=True)[:5],2)
                lists.append(getListDisplayText(header=('Timestamp',group,sort,''), data=data))
            else:
                data=BarPlot.appendBarPlot(t.get(('Timestamp',sort),filterFunc=lambda x:True, sortKey=lambda x:float(x(sort)), reverse=True)[:5],1)
                lists.append(getListDisplayText(header=('Timestamp',sort,''), data=data))
    return lists

In [13]:
with open('../conf/sar.json','r') as jsontst:
    jsonfields=json.load(jsontst)

In [14]:
reportsDict={}
for d in jsonfields['fields']:
    reportName=d.get('report','Others')
    reportsDict[reportName]=reportsDict.get(reportName,[])+[d]

In [15]:
for n,s in fileDictionary.items():
    print(f'Analyzing: {n}:')
    for report in reportsDict.items():
        lists=analyzeFields(report[1], s)
        if len(lists) > 0:
            print(Stage('Analyzing '+report[0]).stringValue())
            stack(*lists, padding=20, lineCapacity=200)

Analyzing: ./saru.txt:

=          Analyzing Memory       =

Timestamp     %memused                                     Timestamp     kbmemused                                 Timestamp     %commit                                     
---------     --------                                     ---------     ---------                                 ---------     -------                                     
19:35:01      78.27        ▄▄▄▄▄▄▄                         19:35:01      38380936      ▄▄▄                         23:45:01      70.07       ▄▄▄▄▄▄▄                         
19:30:01      78.18        ▄▄▄▄▄▄▄                         19:30:01      38337740      ▄▄▄                         23:55:01      70.07       ▄▄▄▄▄▄▄                         
19:25:01      78.12        ▄▄▄▄▄▄▄                         19:25:01      38306944      ▄▄▄                         23:25:01      70.06       ▄▄▄▄▄▄▄                         
19:20:01      78.02        ▄▄▄▄▄▄▄                         19:20:01  